In [1]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket='sagemaker-tagger'
data_key1 = 'Tagged_emails.csv'
data_location1 = 's3://{}/{}'.format(bucket, data_key1)

tagged = pd.read_csv(data_location1)

In [2]:
tagged.head()

,Unnamed: 0,Content-Type,From,Body,Subject,UID,Tags
0,0,multipart/related,John Morrison <John.Morrison@availity.com>,[<email.message.Message object at 0x0000022BB5...,FW: Welcome to Availity!!!,1036,"Personal, PRoductivity"
1,1,multipart/alternative,"""Gendron, Tania, Ph.D."" <Gendron.Tania@mayo.edu>",SGkgSm9obiwNCg0KSSBiZWxpZXZlIHRoYXQgQ2FpdGx5bi...,RE: Special Project's Associate position,1037,"Personal, Productivity"
2,2,multipart/mixed,Kramer Lisa - Brooksville <Lisa.Kramer2@hcahea...,[<email.message.Message object at 0x0000022BB5...,NaN,1038,"Personal, Other"
3,3,multipart/alternative,Chase <Chase@e.chase.com>,"To view the html version of this message, plea...",Tax forms are available online,1039,Finance
4,4,text/plain,GreenEmployee.com Notifier <noreply@greenemplo...,QSBuZXcgcGF5c3R1YiBpcyBub3cgYXZhaWxhYmxlIG9ubG...,Paystub Now Available Online,1040,Finance


In [8]:
def imap_login(address=None, password=None):
    '''Wrapper for logging into to email through IMAP
    
    ARGS: 
    address - str (defaul: None, prompt input). Email address 
    being connected to.
    
    password - str (default: None, prompt input). Password for email address.
    
    Returns:
    Mail object connected to corresponding server for email address'''
    
    import imaplib
    
    if not address:
        address = input('Enter you email address: ')
        
    if not password:
        password = input('Enter your password')
        
    if 'gmail' in address:
        SMTP_SERVER = 'imap.gmail.com'
    elif 'yahoo' in password:
        SMTP_SERVER = 'imap.mail.yahoo.com'
    else:
        raise NameError('Please enter a gmail or yahoo email address')
        
    SMTP_PORT = 993
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(address, password)
    except Exception as e:
        raise
        
    return mail

In [9]:
def search_mailbox(mail, inbox='inbox'):
    """Connects to mailbox and collects a list of ids from mailbox
    
    ARGS:
    mail - logged in mail object
    
    inbox - str (defauls: 'inbox'). Mailbox to connect to. Must be valid
    imap mailbox.
    
    Returns:
    tup (mail object, list of mail_ids)
    If you don't need the ids, you can use an underscore like so:
    mail, _ = search_mailbox(mail)"""
    
    import imaplib
    
    mail.select(inbox)
    
    typ, data = mail.search(None, 'ALL')
    
    mail_ids = data[0].decode()
    mail_ids = mail_ids.split()
    
    return mail, mail_ids

In [21]:
def save_mail(mail, i_d, filename='email_data.csv', verbose=False):
    """Writes email data to csv
    
    ARGS: 
    mail - logged in mail object
    
    i_d - list of i_ds
    ids of messages to get
    
    filename - string ending in .csv (default: 'email_data.csv')
    name of file to write to 
    
    Returns: None, saves data to csv"""
    
    import imaplib
    import email
    import csv
    
    csv_file = open(filename, 'w', encoding='UTF-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['id', 'uid', 'from_', 'subject', 'msg', 'content_type'])
        
    rows = {}
    
    for i in i_d:
        try:
            typ, data = mail.fetch(str(i).encode(), '(UID RFC822)')

            uid = email.message_from_bytes(data[0][0])
            uid = uid.get_payload()
            uid = uid.split()[-3]

            meta = email.message_from_bytes(data[0][1])
            from_ = meta['From']
            subject = meta['Subject']
            content_type = meta['Content-Type'].split(';')[0]
            
            msg = meta.get_payload()
            if type(msg) == list:
                if len(msg) == 3:
                    msg = msg[0].get_payload()[0].get_payload()
                else:
                    msg = msg[0].get_payload()
            
#             print(i)
            if verbose:
                print('UID: ', uid)
                print('From: ', from_)
                print('Subject: ', subject)
                print('Content-Type: ', content_type)
                print('Message: ', msg)
            csv_writer.writerow([i, uid, from_, subject, msg, content_type])
#             print('Message saved')
            
            row = ({'UID': i, 'From': from_, 'Subject': subject, 'Content-Type':content_type, 'Message':msg})
            rows[i] = row
            
        except Exception as e:
            print(e)
        
    df = pd.DataFrame.from_dict(rows)
    return df.T


In [8]:
def clean_emails(df, drop_html=True):
    """Cleans our emails from csv (assumes columns of save_mail func)
    
    ARGS: pandas dataframe
    dataframe from csv with columns of save_mail()
    
    Returns:
    Dataframe with emails cleaned up"""
    
    df['Content-Type'] = df['Content-Type'].map(lambda x: 'multipart/alternative' if x == 'Multipart/Alternative' else x)
    
    if drop_html:
        mask = df['Body'].str.startswith('<')
        df = df[~mask]
        
    
    return df

In [38]:
# Load in email and password from dotenv


email = 'jjosephmorrison@gmail.com'
password = 'Captainmorgan17'

In [41]:
# Login to mail and search mailbox and retrieve mail_ids

mail = imap_login(address=email, password=password)
mail, mail_ids = search_mailbox(mail)

error: b'[ALERT] Please log in via your web browser: https://support.google.com/mail/accounts/answer/78754 (Failure)'

In [22]:
# Create dataframe of emails

df = save_mail(mail, mail_ids)
print(df.shape)
df.head()

(90, 5)


,Content-Type,From,Message,Subject,UID
1,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type], [Content-Type, Content-Transf...",RE: [Non-DoD Source] Re: Air Guard,1
2,text/html,turbotax@intuit.com,<!doctype html>\r\n<html>\r\n <head>\r\n <...,TurboTax Update: Federal Return Accepted,2
3,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type], [Content-Type, Content-Transf...",RE: [Non-DoD Source] Re: Air Guard,3
4,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type], [Content-Type, Content-Transf...",RE: [Non-DoD Source] Info about open position,4
5,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type, Content-Transfer-Encoding], [C...",RE: [Non-DoD Source] Info about open position,5


In [23]:
df = df.rename(columns={'Message': 'Body'})
df.head()

,Content-Type,From,Body,Subject,UID
1,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type], [Content-Type, Content-Transf...",RE: [Non-DoD Source] Re: Air Guard,1
2,text/html,turbotax@intuit.com,<!doctype html>\r\n<html>\r\n <head>\r\n <...,TurboTax Update: Federal Return Accepted,2
3,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type], [Content-Type, Content-Transf...",RE: [Non-DoD Source] Re: Air Guard,3
4,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type], [Content-Type, Content-Transf...",RE: [Non-DoD Source] Info about open position,4
5,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type, Content-Transfer-Encoding], [C...",RE: [Non-DoD Source] Info about open position,5


In [36]:
df.head()

,Content-Type,From,Body,Subject,UID
1,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type], [Content-Type, Content-Transf...",RE: [Non-DoD Source] Re: Air Guard,1
2,text/html,turbotax@intuit.com,<!doctype html>\r\n<html>\r\n <head>\r\n <...,TurboTax Update: Federal Return Accepted,2
3,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type], [Content-Type, Content-Transf...",RE: [Non-DoD Source] Re: Air Guard,3
4,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type], [Content-Type, Content-Transf...",RE: [Non-DoD Source] Info about open position,4
5,multipart/signed,"""Brown, Joshua J MSgt USAF (US)"" <joshua.j.bro...","[[Content-Type, Content-Transfer-Encoding], [C...",RE: [Non-DoD Source] Info about open position,5


In [37]:
df2.head()

,keys,from/to,subject,body,tags,New
0,59,Google <no-reply@accounts.google.com>,App password created,--00000000000002379205972971fd Content-Type: t...,['Security'],Productivity
1,57,Google <no-reply@accounts.google.com>,2-Step Verification turned on,--000000000000d50ed3059729649c Content-Type: t...,Security'],Productivity
2,56,"""Medium Daily Digest"" <noreply@medium.com>",How To Wake Up at 5 A.M. Every Day | Bryan Ye ...,--337b1df667177cb513899ad44e31fb8d4f8637575996...,"['Promotions', 'Productivity']",Entertainment
3,55,"""no-reply-aws@amazon.com"" <no-reply-aws@amazon...",RE:[CASE 6570793521] Limit Increase: SageMaker,------=_Part_26869745_214056034.1573352866590 ...,['Productivity'],Productivity
4,54,Amazon Web Services <aws-marketing-email-repli...,Thank you for attending AWS Machine Learning W...,------=_Part_-1563787701_1894373630.1573225596...,"['Events', 'Productivity']","Events, Productivity"


In [ ]:
# df2 = pd.read_csv('Email_data/John_gmail - Sheet1.csv')
# df2['Tags'] = df2['Tags'].apply(str)
# df2 = df2.drop(columns = ['Body', 'From', 'Subject', 'Unnamed: 0'])
# df2 = df2[df2['Tags'] != 'X']
# print(df2.shape)
# df2.sort_values(by='UID', ascending=True).head()

In [34]:
df2 = df2.rename(columns={'keys':'UID', 'from/to':'From', 'subject':'Subject', 'body':})

(52, 6)

In [35]:
tagged_emails = pd.merge(df, df2)
print(tagged_emails.shape)
tagged_emails.head()


MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False